In [1]:
import os
from pytube import YouTube
from pytube.exceptions import PytubeError
import ffmpeg

In [2]:
def initialize_youtube(url):
    try:
        yt = YouTube(url)
        return yt
    except (PytubeError, Exception) as e:
        print(f"Error initializing YouTube: {e}")
        return None

In [3]:
def download_audio(yt, audio_folder_path):
    try:
        audio_stream = yt.streams.filter(only_audio=True).first()
        if not audio_stream:
            print("No audio stream found")
            return None

        video_title = yt.title
        filename = f"{video_title}.wav"
        full_path = os.path.join(audio_folder_path, filename)

        if os.path.exists(full_path):
            print(f"{filename} already exists. Skipping download.")
        else:
            print(f"Downloading audio: {filename}")
            audio_stream.download(output_path=audio_folder_path, filename=filename)
            print("Download complete")

        return full_path

    except (PytubeError, Exception) as e:
        print(f"Error downloading audio: {e}")
        return None

In [4]:
def trim_audio(input_file, start_time, end_time, output_folder):
    try:
        video_title = os.path.splitext(os.path.basename(input_file))[0]
        start_time_sec = sum(int(x) * 60 ** i for i, x in enumerate(reversed(start_time.split(':'))))
        end_time_sec = sum(int(x) * 60 ** i for i, x in enumerate(reversed(end_time.split(':'))))

        trimmed_filename = f"trimmed_{start_time}-{end_time}_{video_title}.wav"
        trimmed_path = os.path.join(output_folder, trimmed_filename)

        stream = ffmpeg.input(input_file)
        audio = stream.audio
        trimmed_audio = audio.filter('atrim', start=start_time_sec, end=end_time_sec)
        trimmed_audio.output(trimmed_path).run()

        print(f"Trimmed audio saved as {trimmed_path}")
        return trimmed_filename

    except Exception as e:
        print(f"Error trimming audio: {e}")
        return None

In [5]:
def main():
    youtube_url = "https://www.youtube.com/embed/CSxBtZPfzfg"
    start_time = "07:35"
    end_time = "19:50"
    audio_folder = "../Test"

    yt = initialize_youtube(youtube_url)
    if yt:
        full_audio_file = download_audio(yt, audio_folder)
        if full_audio_file:
            trimmed_file = trim_audio(full_audio_file, start_time, end_time, audio_folder)
            if trimmed_file:
                print(f"Downloaded audio: {trimmed_file}")

if __name__ == "__main__":
    main()

Download complete


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopen

Trimmed audio saved as ../Test/trimmed_07:35-19:50_PM Modi addresses Public Meeting at Indore, Madhya Pradesh.wav
Downloaded audio: trimmed_07:35-19:50_PM Modi addresses Public Meeting at Indore, Madhya Pradesh.wav


[out#0/wav @ 0x13680f060] video:0kB audio:126615kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000060%
size=  126615kB time=00:19:49.97 bitrate= 871.6kbits/s speed= 478x    
